<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Feature-Selection" data-toc-modified-id="Feature-Selection-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Feature Selection</a></span><ul class="toc-item"><li><span><a href="#Variance-Thresholding" data-toc-modified-id="Variance-Thresholding-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Variance Thresholding</a></span></li><li><span><a href="#Anova-Test" data-toc-modified-id="Anova-Test-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Anova Test</a></span></li><li><span><a href="#RFE-(Recursive-Feature-Elimination)" data-toc-modified-id="RFE-(Recursive-Feature-Elimination)-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>RFE (Recursive Feature Elimination)</a></span></li><li><span><a href="#Feature-Selection-using-RandomForest" data-toc-modified-id="Feature-Selection-using-RandomForest-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Feature Selection using RandomForest</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Conclusion</a></span></li></ul></li><li><span><a href="#Modelling" data-toc-modified-id="Modelling-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Modelling</a></span><ul class="toc-item"><li><span><a href="#Decision-tree-modelling" data-toc-modified-id="Decision-tree-modelling-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Decision tree modelling</a></span><ul class="toc-item"><li><span><a href="#Hyperparameter-tuning-the-decision-tree" data-toc-modified-id="Hyperparameter-tuning-the-decision-tree-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Hyperparameter tuning the decision tree</a></span></li></ul></li><li><span><a href="#Modelling-with-Random-Forest" data-toc-modified-id="Modelling-with-Random-Forest-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Modelling with Random Forest</a></span><ul class="toc-item"><li><span><a href="#Tuning-the-Random-Forest" data-toc-modified-id="Tuning-the-Random-Forest-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Tuning the Random Forest</a></span></li></ul></li><li><span><a href="#Saving-both-the-models-and-the-results" data-toc-modified-id="Saving-both-the-models-and-the-results-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Saving both the models and the results</a></span></li></ul></li></ul></div>

<center><h1>Feature Selection</h1></center>

In [52]:
import numpy as np
import pandas as pd

from sklearn.experimental import enable_halving_search_cv

from sklearn.preprocessing import (
    MinMaxScaler, 
    StandardScaler
)

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics

from sklearn.model_selection import (
    train_test_split, 
    KFold, 
    HalvingGridSearchCV
)

from sklearn.feature_selection import (
    VarianceThreshold, 
    f_classif,
    SelectKBest,
    SelectFromModel,
    RFE
)

from skopt import BayesSearchCV

from joblib import dump, load

from prettytable import PrettyTable

In [58]:
df = pd.read_csv("./cleaned_data/Data_removed_outlier_iqr.csv")

In [3]:
df.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,35280,717.703,264.995250,170.035245,1.558472,0.766994,35772,211.943132,0.703616,0.986246,0.860694,0.799800,0.007511,0.001896,0.639680,0.996923,DERMASON
1,83296,1142.638,446.765889,239.013317,1.869209,0.844861,84270,325.662035,0.702588,0.988442,0.801709,0.728932,0.005364,0.000934,0.531342,0.993190,CALI
2,35594,689.634,254.572928,178.441837,1.426644,0.713214,35966,212.884213,0.811629,0.989657,0.940479,0.836241,0.007152,0.002157,0.699298,0.997650,DERMASON
3,52710,872.700,326.039383,207.399450,1.572036,0.771592,53280,259.060720,0.677419,0.989302,0.869707,0.794569,0.006186,0.001521,0.631340,0.992488,SIRA
4,62855,1004.759,413.879306,194.299306,2.130112,0.882954,63781,282.894807,0.598340,0.985482,0.782395,0.683520,0.006585,0.000887,0.467200,0.995188,HOROZ


In [4]:
X = df.drop('Class', axis=1)
y = df['Class'].astype('category').cat.codes

In [5]:
X.var(numeric_only=True)

Area               2.822575e+08
Perimeter          2.552720e+04
MajorAxisLength    4.481631e+03
MinorAxisLength    8.885643e+02
AspectRation       6.266115e-02
Eccentricity       8.710692e-03
ConvexArea         2.930358e+08
EquivDiameter      1.712622e+03
Extent             2.414141e-03
Solidity           2.150092e-05
roundness          3.647966e-03
Compactness        3.917317e-03
ShapeFactor1       9.303870e-07
ShapeFactor2       3.371308e-07
ShapeFactor3       1.007840e-02
ShapeFactor4       1.858476e-05
dtype: float64

In [6]:
scaler = MinMaxScaler(feature_range=(1, 10))
X_scaled = scaler.fit_transform(X)

In [7]:
X_scaled_df = pd.DataFrame(X_scaled, columns=df.columns[:-1].tolist())

In [8]:
X_scaled_df.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,2.098625,3.016910,3.087722,2.683622,4.417042,8.122872,2.109502,2.727128,5.293329,8.994033,7.665143,5.132964,6.251251,4.536541,4.656919,9.514172
1,5.648529,7.458373,7.750059,5.127365,6.406905,9.134902,5.675823,6.601082,5.263568,9.256008,6.605667,3.293449,3.512836,1.566158,2.929442,8.868550
2,2.121839,2.723530,2.820394,2.981450,3.572851,7.423900,2.123768,2.759187,8.420327,9.400973,9.098218,6.078862,5.793397,5.344295,5.607558,9.639797
3,3.387254,4.636954,4.653477,4.007355,4.503898,8.182628,3.396961,4.332238,4.534940,9.358605,7.827025,4.997184,4.560889,3.378249,4.523934,8.747308
4,4.137291,6.017248,6.906533,3.543245,8.077647,9.629996,4.169156,5.144171,2.245580,8.902799,6.258752,2.114683,5.069827,1.419470,1.906669,9.214072


In [9]:
X_scaled_df.var()

Area               1.542790
Perimeter          2.788754
MajorAxisLength    2.948464
MinorAxisLength    1.115265
AspectRation       2.569564
Eccentricity       1.471410
ConvexArea         1.584579
EquivDiameter      1.987491
Extent             2.023309
Solidity           0.306082
roundness          1.176912
Compactness        2.639371
ShapeFactor1       1.512696
ShapeFactor2       3.215439
ShapeFactor3       2.562470
ShapeFactor4       0.555753
dtype: float64

## Variance Thresholding

If the variance is low or close to zero, then a feature is approximately constant and will not improve the performance of the model. In that case, it should be removed.

Variance will also be very low for a feature if only a handful of observations of that feature differ from a constant value.

What we can do is set a threshold and drop features with low variance 

In [10]:
vt = VarianceThreshold(threshold=1)
X_scaled_var_feats = vt.fit_transform(X_scaled)

In [11]:
X_scaled_var_feats_df = pd.DataFrame(
    X_scaled_var_feats, 
    columns=X_scaled_df.columns[vt.get_support(indices=True)]
)

In [12]:
X_scaled_var_feats_df.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3
0,2.098625,3.016910,3.087722,2.683622,4.417042,8.122872,2.109502,2.727128,5.293329,7.665143,5.132964,6.251251,4.536541,4.656919
1,5.648529,7.458373,7.750059,5.127365,6.406905,9.134902,5.675823,6.601082,5.263568,6.605667,3.293449,3.512836,1.566158,2.929442
2,2.121839,2.723530,2.820394,2.981450,3.572851,7.423900,2.123768,2.759187,8.420327,9.098218,6.078862,5.793397,5.344295,5.607558
3,3.387254,4.636954,4.653477,4.007355,4.503898,8.182628,3.396961,4.332238,4.534940,7.827025,4.997184,4.560889,3.378249,4.523934
4,4.137291,6.017248,6.906533,3.543245,8.077647,9.629996,4.169156,5.144171,2.245580,6.258752,2.114683,5.069827,1.419470,1.906669


**Obvservation**

 - As we can see 2 low variance features are removed which are `Solidity` and `ShapeFactor4`

## Anova Test

Analysis of variance (ANOVA) is a statistical technique that is used to check if the means of two or more groups are significantly different from each other. ANOVA checks the impact of one or more factors by comparing the means of different samples. 

If we had categorical variables we would do another test called the $\chi^2$ test. Since we have all numeric features we do the ANOVA test.

In [13]:
anova_filter = SelectKBest(score_func=f_classif, k=8)
anova_filter.fit(X, y)

SelectKBest(k=8)

In [14]:
anova_df = pd.DataFrame({
    'features': df.columns[anova_filter.get_support(indices=True)],
    'scores': anova_filter.scores_[anova_filter.get_support(indices=True)]
})

In [15]:
anova_df

,features,scores
0,Area,12106.046095
1,Perimeter,13571.034479
2,MajorAxisLength,13811.838587
3,AspectRation,10508.820302
4,ConvexArea,12251.935448
5,EquivDiameter,12284.518500
6,Compactness,10309.663853
7,ShapeFactor2,10833.619482


**Obvservation**

 - We have selected 8 best features according to the anova scores. 
 - We can select more features also, it can be a thought of as a hyperparameter to be tuned i.e trying different models with different number of features
 - I selected the number 8 as in the previous EDA notebook we saw that 8 principal components could be used to describe the whole dataset

## RFE (Recursive Feature Elimination)

Recursive Feature Elimination selects features by recursively considering smaller subsets of features by pruning the least important feature at each step. Here models are created iteartively and in each iteration it determines the best and worst performing features and this process continues until all the features are explored.Next ranking is given on eah feature based on their elimination orde. In the worst case, if a dataset contains N number of features RFE will do a greedy search for $N^2$ combinations of features.

In [16]:
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=8, verbose=3)
rfe.fit(X, y)

Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.


RFE(estimator=DecisionTreeClassifier(), n_features_to_select=8, verbose=3)

In [17]:
rfe.get_feature_names_out()

array(['Perimeter', 'MajorAxisLength', 'MinorAxisLength', 'roundness',
       'Compactness', 'ShapeFactor1', 'ShapeFactor3', 'ShapeFactor4'],
      dtype=object)

In [18]:
rfe_df = pd.DataFrame(
    {
        'features': X.columns.to_list(),
        'rank': rfe.ranking_,
        'selected': rfe.support_
    }
)

In [19]:
rfe_df

,features,rank,selected
0,Area,3,False
1,Perimeter,1,True
2,MajorAxisLength,1,True
3,MinorAxisLength,1,True
4,AspectRation,8,False
5,Eccentricity,7,False
6,ConvexArea,5,False
7,EquivDiameter,9,False
8,Extent,4,False
9,Solidity,2,False


**Obvservation**

 - We can see that the ones which are marked as `True` are the features selected.

## Feature Selection using RandomForest

Feature selection using Random forest comes under the category of Embedded methods. Embedded methods combine the qualities of filter and wrapper methods. They are implemented by algorithms that have their own built-in feature selection methods. Some of the benefits of embedded methods are :

1. They are highly accurate.
2. They generalize better.
3. They are interpretable

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [21]:
selector = SelectFromModel(estimator=RandomForestClassifier(n_estimators=100, verbose=1, n_jobs=-1))
selector.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.5s finished


SelectFromModel(estimator=RandomForestClassifier(n_jobs=-1, verbose=1))

In [22]:
print(f"Selected features are: {selector.get_feature_names_out()}")

Selected features are: ['Perimeter' 'MajorAxisLength' 'MinorAxisLength' 'ConvexArea'
 'Compactness' 'ShapeFactor1' 'ShapeFactor3']


## Conclusion

I now enlist all the methods used and the features selected

In [23]:
methods = [
    'Variance Thresholding (threshold = 1)',
    'ANOVA F-test',
    'Recursive Feature Elimination (estimator = DecisionTreeClassifier)',
    'Using RandomForest feature importance'
]

features = [
    X_scaled_var_feats_df.columns.to_list(),
    df.columns[anova_filter.get_support(indices=True)].to_list(),
    rfe.get_feature_names_out().tolist(),
    selector.get_feature_names_out().tolist()
]

In [24]:
table = PrettyTable(['Methods', 'Features Selected'])
for m, f in zip(methods, features):
    table.add_row([m, f])

In [25]:
table

Methods,Features Selected
Variance Thresholding (threshold = 1),"['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength', 'AspectRation', 'Eccentricity', 'ConvexArea', 'EquivDiameter', 'Extent', 'roundness', 'Compactness', 'ShapeFactor1', 'ShapeFactor2', 'ShapeFactor3']"
ANOVA F-test,"['Area', 'Perimeter', 'MajorAxisLength', 'AspectRation', 'ConvexArea', 'EquivDiameter', 'Compactness', 'ShapeFactor2']"
Recursive Feature Elimination (estimator = DecisionTreeClassifier),"['Perimeter', 'MajorAxisLength', 'MinorAxisLength', 'roundness', 'Compactness', 'ShapeFactor1', 'ShapeFactor3', 'ShapeFactor4']"
Using RandomForest feature importance,"['Perimeter', 'MajorAxisLength', 'MinorAxisLength', 'ConvexArea', 'Compactness', 'ShapeFactor1', 'ShapeFactor3']"


# Modelling

Let's start with features selected using the ANOVA F-test

Our metric of choice is going to be `Weighted F1-score`. 
Here's a very good explanation for it: [link](https://stats.stackexchange.com/questions/463224/which-performance-metrics-for-highly-imbalanced-multiclass-dataset) 

In [26]:
feats = anova_filter.get_feature_names_out().tolist()

In [27]:
X[feats].head(10)

,Area,Perimeter,MajorAxisLength,AspectRation,ConvexArea,EquivDiameter,Compactness,ShapeFactor2
0,35280,717.703,264.995250,1.558472,35772,211.943132,0.799800,0.001896
1,83296,1142.638,446.765889,1.869209,84270,325.662035,0.728932,0.000934
2,35594,689.634,254.572928,1.426644,35966,212.884213,0.836241,0.002157
3,52710,872.700,326.039383,1.572036,53280,259.060720,0.794569,0.001521
4,62855,1004.759,413.879306,2.130112,63781,282.894807,0.683520,0.000887
5,36112,723.077,225.328776,1.100957,36709,214.427672,0.951621,0.003156
6,59442,975.979,402.887444,2.118794,60289,275.107079,0.682839,0.000909
7,58931,957.164,388.354344,2.001383,59526,273.922032,0.705340,0.001006
8,34010,681.989,257.829640,1.528818,34410,208.093433,0.807097,0.001984
9,27280,603.203,223.021086,1.428199,27550,186.370531,0.835663,0.002459


In [62]:
anova = pd.concat([X[feats], y], axis=1)

In [69]:
anova.rename(columns={anova.columns[-1]: 'labels'}, inplace=True)

In [70]:
# anova.to_csv('./selected_features/Anova.csv', index=False)

In [28]:
X_ = X[feats]
y_ = y

In [29]:
def cross_validate(X, y, model):
    kfold = KFold(n_splits=10)
    idx = 1
    for train_index, test_index in kfold.split(X=X, y=y):
        X_train_ , X_test_ = X.iloc[train_index,:],X.iloc[test_index,:]
        y_train_ , y_test_ = y[train_index] , y[test_index]
        
        model.fit(X_train_, y_train_)
        y_pred_ = model.predict(X_test_)
            
        print(
            f"[FOLD {idx}] " 
            f"Weighted F1-score: {round(metrics.f1_score(y_pred_, y_test_, average='weighted'), 3)} "
        )
        
        idx += 1

## Decision tree modelling

In [30]:
X_train_, X_test_, y_train_, y_test_ = train_test_split(X_, y_, test_size=0.3)

dt = DecisionTreeClassifier()
dt.fit(X_train_, y_train_)

preds = dt.predict(X_test_)
print(f"Prediction from the baseline model, F1-score: {metrics.f1_score(preds, y_test_, average='weighted')}")

Prediction from the baseline model, F1-score: 0.8752931788513145


### Hyperparameter tuning the decision tree

A very good guide to tuning the decision tree and it's various parameters I refered to is: [How to tune a Decision Tree ?](https://towardsdatascience.com/how-to-tune-a-decision-tree-f03721801680)

In [31]:
params_dt = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'max_features': ['sqrt', 'auto', 'log2'],
    'criterion': ["gini", "entropy"],
    'splitter': ['best', 'random']
}

In [32]:
scorer = metrics.make_scorer(
    metrics.f1_score, average='weighted'
)

hf_gs_cv_dt = HalvingGridSearchCV(
    estimator=DecisionTreeClassifier(), n_jobs=-1,
    param_grid=params_dt, scoring=scorer, cv=10, verbose=10, 
)

In [34]:
%%time
hf_gs_cv_dt.fit(X_train_, y_train_)

n_iterations: 4
n_required_iterations: 6
n_possible_iterations: 4
min_resources_: 140
max_resources_: 9177
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 300
n_resources: 140
Fitting 10 folds for each of 300 candidates, totalling 3000 fits
----------
iter: 1
n_candidates: 100
n_resources: 420
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
----------
iter: 2
n_candidates: 34
n_resources: 1260
Fitting 10 folds for each of 34 candidates, totalling 340 fits
----------
iter: 3
n_candidates: 12
n_resources: 3780
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Wall time: 9.14 s


HalvingGridSearchCV(cv=10, estimator=DecisionTreeClassifier(), n_jobs=-1,
                    param_grid={'criterion': ['gini', 'entropy'],
                                'max_depth': [2, 3, 5, 10, 20],
                                'max_features': ['sqrt', 'auto', 'log2'],
                                'min_samples_leaf': [5, 10, 20, 50, 100],
                                'splitter': ['best', 'random']},
                    scoring=make_scorer(f1_score, average=weighted),
                    verbose=10)

In [35]:
hf_gs_cv_dt.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 'log2',
 'min_samples_leaf': 5,
 'splitter': 'best'}

In [36]:
hf_gs_cv_dt_df = pd.DataFrame(hf_gs_cv_dt.cv_results_)
hf_gs_cv_dt_df.head()

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_max_features,param_min_samples_leaf,...,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
0,0,140,0.003553,0.000654,0.003801,0.001167,gini,2,sqrt,5,...,0.546654,0.518397,0.668809,0.489652,0.613769,0.515568,0.580307,0.581991,0.568230,0.052120
1,0,140,0.004199,0.001470,0.003801,0.000600,gini,2,sqrt,5,...,0.394000,0.523475,0.436813,0.345508,0.456166,0.346716,0.300636,0.601299,0.406380,0.094296
2,0,140,0.004900,0.003805,0.003300,0.000457,gini,2,sqrt,10,...,0.407434,0.527978,0.682341,0.422399,0.460875,0.692151,0.490465,0.553256,0.537019,0.103310
3,0,140,0.003301,0.000640,0.003100,0.000539,gini,2,sqrt,10,...,0.445844,0.520801,0.344661,0.263836,0.395119,0.424009,0.390370,0.166667,0.344682,0.118562
4,0,140,0.003501,0.000671,0.003200,0.000400,gini,2,sqrt,20,...,0.583193,0.742487,0.487347,0.633832,0.537617,0.498577,0.490465,0.511854,0.543819,0.082293


In [37]:
hf_gs_cv_dt_df.nlargest(5, columns=['mean_test_score'])

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_max_features,param_min_samples_leaf,...,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
439,3,3780,0.029001,0.003006,0.0031,0.000301,entropy,10,log2,5,...,0.933640,0.932294,0.924811,0.918196,0.920690,0.912110,0.917584,0.920323,0.923591,0.006813
437,3,3780,0.014499,0.001025,0.0037,0.001487,gini,10,log2,5,...,0.918968,0.915394,0.915961,0.926768,0.920363,0.917944,0.923957,0.925481,0.919858,0.004416
422,2,1260,0.006299,0.000641,0.0029,0.000539,gini,20,log2,5,...,0.921873,0.925950,0.917111,0.934911,0.919555,0.924250,0.922334,0.926750,0.924553,0.004933
443,3,3780,0.021899,0.001376,0.0030,0.000774,entropy,20,sqrt,5,...,0.934757,0.933998,0.931156,0.930058,0.926677,0.924585,0.930666,0.927944,0.929980,0.002967
442,3,3780,0.012001,0.001485,0.0030,0.000004,gini,10,sqrt,5,...,0.923349,0.916846,0.917335,0.913921,0.917590,0.911338,0.921140,0.923916,0.918377,0.004521


In [38]:
best_model = hf_gs_cv_dt.best_estimator_

Fitting the best estimator after tuning

In [39]:
preds = best_model.predict(X_test_)
print(f"Prediction from the best model, F1-score: {metrics.f1_score(preds, y_test_, average='weighted')}")

Prediction from the best model, F1-score: 0.8837836323045146


**Obvservation**

- After hyperparamter tuning we can clearly see a improvement in our model without any under-sampling or over-sampling
- With the default `DecisionTreeClassifier` we get approx ~ 0.86 weighted f1-score.
- After tuning we are now getting approx ~ 0.89 weighted f1-score.

## Modelling with Random Forest

In [40]:
dt = RandomForestClassifier(n_jobs=-1, verbose=1)
dt.fit(X_train_, y_train_)

preds = dt.predict(X_test_)
print(f"Prediction from the baseline model, F1-score: {metrics.f1_score(preds, y_test_, average='weighted')}")

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s


Prediction from the baseline model, F1-score: 0.9078855725000966


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished


### Tuning the Random Forest

We have already seen `HalvingGridSearchCV` for tuning the decision tree. We will now use `BayesSearchCV` which is a better way and more efficient way to tune the hyper parameters rather than just trying every possible combination. 

The one-sentence summary of Bayesian hyperparameter optimization is: build a probability model of the objective function and use it to select the most promising hyperparameters to evaluate in the true objective function.

Detailed explanation in: [A Conceptual Explanation of Bayesian Hyperparameter Optimization for Machine Learning](https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f)

In [43]:
params_rf = {
    'bootstrap': [True, False],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [100, 200, 400, 600, 800, 1000]
}

In [44]:
bs_cv_rf = BayesSearchCV(
    estimator=RandomForestClassifier(n_jobs=-1), 
    search_spaces=params_rf, n_iter=20,
    cv=5, verbose=2, scoring=scorer
)

In [45]:
%time
bs_cv_rf.fit(X_, y_)

Wall time: 0 ns
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=400; total time=   1.5s
[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=400; total time=   1.4s
[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=400; total time=   1.4s
[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=400; total time=   1.4s
[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=400; total time=   1.4s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=800; total time=   3.3s
[CV] END bootstrap=True, max_depth=20, max_feature

[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=400; total time=   2.6s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=400; total time=   2.5s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=400; total time=   2.5s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=400; total time=   2.6s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=400; total time=   2.6s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END bootstrap=True, max_depth=80, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=800; total time=   3.2s
[CV] END bootstrap=True, max_depth=80, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=8

BayesSearchCV(cv=5, estimator=RandomForestClassifier(n_jobs=-1), n_iter=20,
              scoring=make_scorer(f1_score, average=weighted),
              search_spaces={'bootstrap': [True, False],
                             'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90,
                                           100, None],
                             'max_features': ['auto', 'sqrt'],
                             'min_samples_leaf': [1, 2, 4],
                             'min_samples_split': [2, 5, 10],
                             'n_estimators': [100, 200, 400, 600, 800, 1000]},
              verbose=2)

In [51]:
bs_cv_rf.best_params_

OrderedDict([('bootstrap', True),
             ('max_depth', 50),
             ('max_features', 'sqrt'),
             ('min_samples_leaf', 2),
             ('min_samples_split', 10),
             ('n_estimators', 200)])

In [48]:
best_model = bs_cv_rf.best_estimator_

In [50]:
preds = best_model.predict(X_test_)
print(f"Prediction from the tuned model, F1-score: {metrics.f1_score(preds, y_test_, average='weighted')}")

Prediction from the tuned model, F1-score: 0.9611404808283409


**Obvservation**

- After hyperparamter tuning we can clearly see a improvement in our model without any under-sampling or over-sampling
- With the default `RandomForestClassifier` we get approx ~ 0.90 weighted f1-score.
- After tuning we are now getting approx ~ 0.96 weighted f1-score, this is massive!

## Saving both the models and the results

In [54]:
dump(hf_gs_cv_dt.best_estimator_, './ML_models/DecisionTree-tuned.joblib')
dump(bs_cv_rf.best_estimator_, './ML_models/RandomForest-tuned.joblib')

['./ML_models/RandomForest-tuned.joblib']

In [55]:
hf_gs_cv_dt_df.to_csv('./ML_results/DecisionTree_HalvingGridSearchCV_results.csv', index=False)

In [56]:
bs_cv_rf_df = pd.DataFrame(bs_cv_rf.cv_results_)
bs_cv_rf_df.to_csv('./ML_results/RandomForest_BayesSearchCV_results.csv', index=False)